# Byte-Pair Encoding

Dans cette séance, on se propose de réécrire l'algorithme *Byte-Pair encoding* pour améliorer les performances de génération de texte. Nous nous sommes librement inspiré de la [vidéo Youtube](https://www.youtube.com/watch?v=zduSFxRajkE) d'[Andrej Karpathy](https://fr.wikipedia.org/wiki/Andrej_Karpathy) pour réécrire l'algorithme. Puis nous mesurerons le gain de ce travail dans un second temps en utilisant des réseaux de neurones récurrents pour générer du texte.

Commençons par définir une phrase qui nous servira de test pour l'ensemble de la phase de construction de l'algorithme. Elle est issue du roman [Madame Bovary](https://fr.wikipedia.org/wiki/Madame_Bovary) de [Gustave Flaubert](https://fr.wikipedia.org/wiki/Gustave_Flaubert).


In [1]:
string = """La conversation de Charles était plate comme un trottoir de rue, et les idées de tout le monde y défilaient dans leur costume ordinaire, sans exciter d'émotion, de rire ou de rêverie"""
print(string)

La conversation de Charles était plate comme un trottoir de rue, et les idées de tout le monde y défilaient dans leur costume ordinaire, sans exciter d'émotion, de rire ou de rêverie


La manière naïve que l'on peut utiliser pour *découper* un texte est de le séparer caractère par caractère puis d'entraîner un algorithme à prédire le prochain caractère. Nous réaliserons cette méthode, et nous allons la comparer à l'approche tokens.

Chaque caractère peut être transformé en un entier à l'aide de la fonction `ord`. La fonction `chr` renvoie le caractère associé à un entier. 

**Consigne** : Tester la fonction `ord` et la fonction `chr` sur les premiers caractères de la chaîne `string`.


In [2]:
caracters_as_integers = [ord(character) for character in string[:5]]
transcription = [chr(integer) for integer in caracters_as_integers]
print(caracters_as_integers)
print(transcription)

[76, 97, 32, 99, 111]
['L', 'a', ' ', 'c', 'o']


**Question** : Le caractère *L* est-il encodé de la même manière que le caractère *l* ?



In [3]:
for caracter in ["L", "l"]:
    print(f"Le caractère {caracter} est encodé comme {ord(caracter)}")

Le caractère L est encodé comme 76
Le caractère l est encodé comme 108


La méthode `encode` permet de transformer une chaîne de caractères en une liste d'entiers. Ces entiers sont les mêmes que ceux que l'on a obtenus avec la fonction `ord` si on utilise l'encodage *utf-8*. Nous utiliserons ici l'encodage *latin* parce que nous avons des caractères *spéciaux* comme *é* ou *è*.

**Consigne** : Vérifier l'affirmation précédente, puis encoder la totalité de la chaîne `string` dans la variable `tokens`. La méthode `encode` renvoie une chaîne binaire. Il conviendra de la transformer en une liste d'entier.


In [4]:
tokens = string.encode("latin")
tokens = list(map(int, tokens))
print(tokens)

[76, 97, 32, 99, 111, 110, 118, 101, 114, 115, 97, 116, 105, 111, 110, 32, 100, 101, 32, 67, 104, 97, 114, 108, 101, 115, 32, 233, 116, 97, 105, 116, 32, 112, 108, 97, 116, 101, 32, 99, 111, 109, 109, 101, 32, 117, 110, 32, 116, 114, 111, 116, 116, 111, 105, 114, 32, 100, 101, 32, 114, 117, 101, 44, 32, 101, 116, 32, 108, 101, 115, 32, 105, 100, 233, 101, 115, 32, 100, 101, 32, 116, 111, 117, 116, 32, 108, 101, 32, 109, 111, 110, 100, 101, 32, 121, 32, 100, 233, 102, 105, 108, 97, 105, 101, 110, 116, 32, 100, 97, 110, 115, 32, 108, 101, 117, 114, 32, 99, 111, 115, 116, 117, 109, 101, 32, 111, 114, 100, 105, 110, 97, 105, 114, 101, 44, 32, 115, 97, 110, 115, 32, 101, 120, 99, 105, 116, 101, 114, 32, 100, 39, 233, 109, 111, 116, 105, 111, 110, 44, 32, 100, 101, 32, 114, 105, 114, 101, 32, 111, 117, 32, 100, 101, 32, 114, 234, 118, 101, 114, 105, 101]


Avant de continuer, nous souhaitons avoir une fonction qui va décoder une liste de token en texte lisible.

**Consigne**: Définir la fonction `tokens_to_text` qui prend en paramètre une liste de token et un dictionnaire qui correspond au vocabulaire.

In [5]:
vocabulary = {index: chr(index) for index in range(256)}

def tokens_to_text(tokens, vocabulary):
    text = "".join(map(vocabulary.get, tokens))
    return text

L'algorithme *Byte-Pair encoding* est une méthode de *tokenization* qui consiste à découper un texte en un ensemble de *tokens*. Un *token* est un mot ou une séquence de mots qui sont formés comme un couple de caractères fréquemment adjacent.
Pour être capable d'identifier de tel couple, il faut être capable de calculer pour une liste d'entier la fréquence d'apparition de chaque paire.

**Consigne** : Définir une fonction `get_stats` qui prend en entrée une liste d'entier et qui renvoie un dictionnaire. Le dictionnaire contient les paires d'entiers en tant que clés, et leur nombre d'apparitions en tant que valeurs.

In [6]:
def get_stats(indexes):
    counts = {}
    for pair in zip(indexes, indexes[1:]):
        counts[pair] = counts.get(pair, 0) + 1
    return counts

**Consigne** : Tester la fonction `get_stats` sur la liste `tokens` et stocker le résultat dans une variable `stats`.



In [7]:
stats = get_stats(tokens)
print(stats)

{(76, 97): 1, (97, 32): 1, (32, 99): 3, (99, 111): 3, (111, 110): 4, (110, 118): 1, (118, 101): 2, (101, 114): 3, (114, 115): 1, (115, 97): 2, (97, 116): 2, (116, 105): 2, (105, 111): 2, (110, 32): 2, (32, 100): 8, (100, 101): 6, (101, 32): 11, (32, 67): 1, (67, 104): 1, (104, 97): 1, (97, 114): 1, (114, 108): 1, (108, 101): 4, (101, 115): 3, (115, 32): 5, (32, 233): 1, (233, 116): 1, (116, 97): 1, (97, 105): 3, (105, 116): 2, (116, 32): 4, (32, 112): 1, (112, 108): 1, (108, 97): 2, (116, 101): 2, (111, 109): 1, (109, 109): 1, (109, 101): 2, (32, 117): 1, (117, 110): 1, (32, 116): 2, (116, 114): 1, (114, 111): 1, (111, 116): 2, (116, 116): 1, (116, 111): 2, (111, 105): 1, (105, 114): 3, (114, 32): 3, (32, 114): 3, (114, 117): 1, (117, 101): 1, (101, 44): 2, (44, 32): 3, (32, 101): 2, (101, 116): 1, (32, 108): 3, (32, 105): 1, (105, 100): 1, (100, 233): 2, (233, 101): 1, (111, 117): 2, (117, 116): 1, (32, 109): 1, (109, 111): 2, (110, 100): 1, (32, 121): 1, (121, 32): 1, (233, 102): 1, 

**Consigne** : Exécuter la cellule-ci dessous pour identifier la paire la plus fréquente, puis traduire les trois premières paires en terme de caractères.

In [8]:
stats_sorted = sorted(stats.items(), key=lambda x: x[1], reverse=True)
print(stats_sorted)

[((101, 32), 11), ((32, 100), 8), ((100, 101), 6), ((115, 32), 5), ((111, 110), 4), ((108, 101), 4), ((116, 32), 4), ((32, 99), 3), ((99, 111), 3), ((101, 114), 3), ((101, 115), 3), ((97, 105), 3), ((105, 114), 3), ((114, 32), 3), ((32, 114), 3), ((44, 32), 3), ((32, 108), 3), ((118, 101), 2), ((115, 97), 2), ((97, 116), 2), ((116, 105), 2), ((105, 111), 2), ((110, 32), 2), ((105, 116), 2), ((108, 97), 2), ((116, 101), 2), ((109, 101), 2), ((32, 116), 2), ((111, 116), 2), ((116, 111), 2), ((101, 44), 2), ((32, 101), 2), ((100, 233), 2), ((111, 117), 2), ((109, 111), 2), ((105, 101), 2), ((97, 110), 2), ((110, 115), 2), ((32, 111), 2), ((114, 101), 2), ((114, 105), 2), ((76, 97), 1), ((97, 32), 1), ((110, 118), 1), ((114, 115), 1), ((32, 67), 1), ((67, 104), 1), ((104, 97), 1), ((97, 114), 1), ((114, 108), 1), ((32, 233), 1), ((233, 116), 1), ((116, 97), 1), ((32, 112), 1), ((112, 108), 1), ((111, 109), 1), ((109, 109), 1), ((32, 117), 1), ((117, 110), 1), ((116, 114), 1), ((114, 111), 

In [9]:
for index, pairs in enumerate(stats_sorted[:3]):
    pair = "".join(map(chr, pairs[0]))
    print(f"La paire {index+1} est '{pair}'")

La paire 1 est 'e '
La paire 2 est ' d'
La paire 3 est 'de'


Pour extraire plus simplement la paire la plus fréquente, on peut utiliser la méthode `max` avec la clé `stats.get`.

**Consigne** : Afficher la paire la plus fréquente avec la méthode `max`.

In [10]:
top_pair = max(stats, key=stats.get)
print(top_pair)

(101, 32)


Nous sommes à présent capable d'identifier la paire d'entier la plus fréquente dans une liste d'entier, chacun représentant un *token / caractères*. Nous devons à présent remplacer la paire la plus fréquente par un nouvel index.

**Consigne** : Définir une fonction `merge` qui prend en entrée une liste d'entier, une paire d'entier, et un nouvel index, et qui renvoie une liste d'entier. Cette fonction remplace toutes les occurrences de la paire d'entier par le nouvel index dans la liste d'entier.

In [11]:
def merge(indexes, pair, new_index):
    new_indexes = []
    index = 0
    
    while index < len(indexes):

        if index < len(indexes) - 1 and indexes[index] == pair[0] and indexes[index+1] == pair[1]:
            new_indexes.append(new_index)
            index += 2
        else:
            new_indexes.append(indexes[index])
            index += 1
    
    return new_indexes

**Consigne** : Vérifier le bon fonctionnement de la fonction `merge` avec la liste présente dans la cellule ci-dessous.

In [12]:
print(merge([5, 6, 6, 7, 9, 1], (6, 7), 256))

[5, 6, 256, 9, 1]


L'encoding *utf-8* est un encoding de 32 bits donc il peut représenter 256 différents entiers, numéroté de 0 à 255. Ainsi, nous allons itérativement remplacer dans la chaîne de caractère `string` les paires les plus fréquentes par un nouvel index.

**Consigne** : Définir une fonction `byte_pair_encoding` qui prend en entrée ...


In [13]:
def byte_pair_encoding(tokens, vocabulary, max_vocab_size, starting_new_index=256):
    new_tokens = tokens.copy()
    vocab_size_left = max_vocab_size - len(vocabulary)
    new_index = starting_new_index

    while vocab_size_left > 0:
        stats = get_stats(new_tokens)
        top_pair = max(stats, key=stats.get)
        vocabulary[new_index] = vocabulary[top_pair[0]] + vocabulary[top_pair[1]]
        new_tokens = merge(new_tokens, top_pair, new_index)

        new_index += 1
        vocab_size_left -= 1
    
    return new_tokens, vocabulary

**Consigne** : Utiliser la fonction `byte_pair_encoding` sur la liste `tokens` pour une taille de vocabulaire de 32, puis calculer le pourcentage de réduction de la liste.

In [14]:
new_tokens, new_vocabulary = byte_pair_encoding(tokens, vocabulary, 260)
rate = (len(new_tokens) - len(tokens)) / len(tokens)
print(f"Le pourcentage de réduction est de {rate * 100:.2f}%")

Le pourcentage de réduction est de -15.38%


## Chargement du texte

Il nous reste à maintenant charger puis processer du texte. Commençons par définir une fonction `clean_text` qui prend en paramètre du texte et qui remplace un ensemble de caractère pour simplifier le travail de tokenization. Par exemple :
* 'à' devient 'a'
* 'é' devient 'e'
* 'æ' devient 'ae'

In [15]:
def clean_text(text):
    changements = {
        '{': '', '}': '', '°': ' ', 'ç': 'c', 'â': 'a', 'ù': 'u', 'ë': 'e', '<': '', '>': '', '_': '', 'œ': 'oe', '*': '', '^': '', 'á': 'a', 'ô': 'o',
        'ï': 'i', 'æ': 'ae', 'º': '', '«': '"', '»': '"', '’': "'", 'û': 'u', 'ü': 'u', 'ê': 'e', 'à': 'a', 'î': 'i', 'è': 'e', 'é': 'e'
        }
    
    string = text
    for key, value in changements.items():
        string = string.replace(key, value)
    
    return string

Cette fonction s'applique sur du texte: il faut définir une fonction qui va l'extraire. Les fichiers viennent du projet [Gutenberg](https://gutenberg.org/), ils ont donc un format particulier où l'oeuvre est en entier mais ne commence et ne termine pas par l'oeuvre. Il y a du texte spécifiant le livre, les métadonnées et le projet par exemple. Ces informations ne sont pas utile pour notre travail.

**Consigne**: Définir une fonction `load_text` qui prend en paramètre :
* *filename*: nom du fichier de texte
* *sentence_start*: ligne qui indique le début du texte
* *sentence_end*: ligne qui indique la fin du texte
* *train_ratio*: proportition du ratio du texte qui sera dans le dataset d'entraînement et la proporition du texte dans le dataset de test

Cette fonction devra lire le fichier de texte, et extraire l'ensemble du texte entre les deux lignes spécifiées en paramètre. Elle renvoie le texte complet nettoyé avec la fonction `clean_text` dans deux datasets: un d'entraînement et un de test.


In [16]:
def load_text(filename, sentence_start, sentence_end, train_ratio=0.8):
    start = False
    book = open(filename, 'r' , encoding = 'utf-8')
    lines = book.readlines()
    verses = []

    for line in lines:
        line = line.strip().lower()
        if(sentence_start.lower() in line and start==False):
            start = True
        if(sentence_end.lower() in line):
            break
        if(start==False or len(line) == 0):
            continue
        verses.append(line)

    book.close()
    text = " ".join(verses)
    text = clean_text(text)

    index = int(len(text) * train_ratio)
    text_train, text_test = text[:index], text[index:]
    return text_train, text_test

On se propose de charger des oeuvres puis d'encoder le texte.

In [17]:
train_bovary, test_bovary = load_text(
    filename="MadameBovary.txt",
    sentence_start="Nous étions à l'Étude, quand le Proviseur entra, suivi d'un",
    sentence_end="Il vient de recevoir la croix d'honneur."
)

train_education, test_education = load_text(
    filename="EducationSentimentale.txt",
    sentence_start="Le 15 septembre 1840, vers six heures du matin, la Ville-de-Montereau",
    sentence_end="ça me vengera!»"
)

train_chartreuse, test_chartreuse = load_text(
    filename="Chartreuse.txt",
    sentence_start="Le 15 mai 1796, le général Bonaparte fit son entrée dans Milan à la",
    sentence_end="grands-ducs de Toscane."
)

train_rougenoir, test_rougenoir = load_text(
    filename="RougeNoir.txt",
    sentence_start="La petite ville de Verrières peut passer pour l'une des plus jolies de",
    sentence_end="embrassant ses enfants."
)

text_train = train_bovary + train_education + train_chartreuse + train_rougenoir
text_test = test_bovary + test_education + test_chartreuse + test_rougenoir


vocabulary = {index: caracter for index, caracter in enumerate(set(text_train))}
encode = {item[1]: item[0] for item in vocabulary.items()}
tokens_train = list(map(encode.get, text_train))

Il faut maintenant passer du texte encodé aux tokens à proprement parler. On souhaite travailler avec une taille de vocabulaire de 128.

**Consigne**: Tokenizer le texte d'entraînement puis celui de test. Mesurer également la compression réalisée.

In [19]:
max_vocab_size = 128
tokens_train, vocabulary = byte_pair_encoding(
    tokens=tokens_train,
    vocabulary=vocabulary,
    max_vocab_size=max_vocab_size,
    starting_new_index=len(vocabulary)
)

rate = (len(tokens_train) - len(text_train)) / len(text_train)
print(f"Le pourcentage de réduction est de {rate * 100:.2f}%")
print(f"Tokens: {len(tokens_train)}\nVocabulaire: {len(vocabulary)}")
encode = {item[1]: item[0] for item in vocabulary.items()}
tokens_test = list(map(encode.get, text_test))

Le pourcentage de réduction est de -43.42%
Tokens: 1424056
Vocabulaire: 128


A ce stade nous n'avons que du texte mais pas un dataset pour s'entraîner. La fonction `get_dataset` permet de le faire. On décide de travailler avec une longueur de séquence de 64.

In [23]:
import tensorflow as tf
import numpy as np
import keras


def get_datasets(tokens, sequence_length, stride=1):
    sequences = []
    y_character = []
    for index in range(0, len(tokens) - sequence_length, stride):
        sequences.append(tokens[index:(index + sequence_length)])
        y_character.append(tokens[index + sequence_length])
    
    sequences = keras.utils.pad_sequences(sequences, maxlen=sequence_length)
    X = np.array(sequences)
    y = keras.utils.to_categorical(y_character, num_classes=max_vocab_size)
    return X, y


sequence_length = 64
X_train, y_train = get_datasets(tokens_train, sequence_length)
X_test, y_test = get_datasets(tokens_test, sequence_length)

## Modélisation

Nous avons deux datasets correspondant à du texte tokenize. Il faut maintenant construire le modèle de langage. Pour le faire, on utilisera la couche [Embedding](https://keras.io/api/layers/core_layers/embedding/) en début de modèle pour réduire la dimension du texte, puis on réalise un réseau de neurone récurrent *classique*.

**Consigne**: Définir ce modèle.


In [26]:
model = keras.models.Sequential([
    keras.layers.Embedding(input_dim=max_vocab_size, output_dim=64),
    keras.layers.SimpleRNN(64, return_sequences=True, activation="relu", kernel_initializer="he_normal", unroll=True),
    keras.layers.LayerNormalization(),
    keras.layers.SimpleRNN(64, return_sequences=True, activation="relu", kernel_initializer="he_normal", unroll=True),
    keras.layers.LayerNormalization(),
    keras.layers.SimpleRNN(64, return_sequences=False, activation="relu", kernel_initializer="he_normal", unroll=True),

    keras.layers.Dense(units=32, activation="relu"),
    keras.layers.Dropout(rate=0.25),
    keras.layers.Dense(units=32, activation="relu"),
    keras.layers.Dropout(rate=0.25),
    keras.layers.Dense(units=64, activation="relu"),
    
    keras.layers.Dense(units=max_vocab_size, activation='softmax')
])

**Consigne**: Entraîner le modèle après avoir défini des hyper-paramètres cohérent.